如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数

In [1]:
#init模块，包含多种模型初始化方法。

import torch
from torch import nn
from torch.nn import init

net=nn.Sequential(
    nn.Linear(4,3),
    nn.ReLU(),
    nn.Linear(3,1)
)

print(net)
X=torch.rand(2,4)
print(net(X).size())
Y=net(X).sum()
print(Y)

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)
torch.Size([2, 1])
tensor(1.0375, grad_fn=<SumBackward0>)


## 4.2.1 访问模型参数

Sequential类与Module类的继承关系。对于Sequential实例中含模型参数的层，我们可以通过Module类的parameters()或者named_parameters方法来访问所有参数（以迭代器的形式返回），后者除了返回参数Tensor外还会返回其名字。

In [2]:
#访问多层感知机net的所有参数

print(type(net.named_parameters()))#为什么是generator类型
for name,param in net.named_parameters():#自动加'weight','bias'的name名
    print(name,param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


对于使用Sequential类构造的神经网络，我们可以通过方括号[]来访问网络的任一层。索引0表示隐藏层为Sequential实例最先添加的层。

In [3]:
#访问net中单层的参数

for name,param in net[0].named_parameters():
    print(name,param.size(),type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


<font color='blue'>param的类型为torch.nn.parameter.Parameter，其实这是Tensor的子类，和Tensor不同的是如果一个Tensor是Parameter，那么它会自动被添加到模型的参数列表里。</font>

In [4]:
class MyModel(nn.Module):
    def __init__(self,**kwargs):
        super(MyModel,self).__init__(**kwargs)
        self.weight1=nn.Parameter(torch.rand(20,20))#测试是否被加到模型参数列表中
        self.weight2=torch.rand(20,20)
    
    def forward(self,x):
        pass

n=MyModel()
for name,param in n.named_parameters():
    print(name)

weight1


因为Parameter是Tensor，即Tensor拥有的属性它都有，比如可以根据data来访问参数数值，用grad来访问参数梯度。

In [5]:
weight_0=list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)#反向传播前梯度为None
Y.backward()#为什么不是根据误差进行反向传播
print(weight_0.grad)

tensor([[ 0.4962, -0.3997, -0.4672,  0.1563],
        [ 0.3101,  0.4098, -0.3128, -0.2650],
        [ 0.4419, -0.0384, -0.3100, -0.2552]])
None
tensor([[-0.2597, -0.0914, -0.1951, -0.2985],
        [ 0.0952,  0.0431,  0.0751,  0.0902],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])


## 4.2.2 初始化模型参数

In [6]:
#将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，将偏差参数清零

for name,param in net.named_parameters():
    if 'weight'in name:
        init.normal_(param,mean=0,std=0.01)
        print(name,param.data)

0.weight tensor([[-0.0136,  0.0115, -0.0144,  0.0140],
        [ 0.0184, -0.0013, -0.0006,  0.0176],
        [-0.0006,  0.0092, -0.0066, -0.0019]])
2.weight tensor([[ 0.0080, -0.0057,  0.0037]])


In [7]:
#使用常数初始化偏置参数

for name,param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param,val=0)#设置偏置项为常数项
        print(name,param.data)
        

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


## 4.2.3 自定义初始化方法

In [8]:
# torch.nn.init.normal_初始化方法
def normal_(tensor,mean=0,std=0.01):
    with torch.no_grad():#with 语法用于简化资源操作的后续清除操作
        return tensor.normal_(mean,std)

In [9]:
# 令权重有一半概率初始化为0，
# 有另一半概率初始化为[−10,−5]和[5,10]两个区间里均匀分布的随机数。

def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        tensor*=(tensor.abs()>=5).float()#当绝对值大于5时：tensor*1，小于5时：tensor*0

for name,param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print('name:param\n',name,param)

name:param
 0.weight Parameter containing:
tensor([[-6.2914,  7.6446,  0.0000, -0.0000],
        [ 0.0000,  0.0000, -0.0000, -9.9715],
        [ 6.6579,  7.6042,  5.7890, -0.0000]], requires_grad=True)
name:param
 2.weight Parameter containing:
tensor([[-0., 0., -0.]], requires_grad=True)


In [10]:
#还可通过改变这些参数的data来改写模型参数值，同时不会影响梯度

for name,param in net.named_parameters():
    if 'bias' in name:
        param.data +=1
        print(name,param.data)


0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


## 4.2.4 共享模型参数


如何共享模型参数: Module类的forward函数里多次调用同一个层。此外，如果我们传入Sequential的模块是同一个Module实例的话参数也是共享的。

In [11]:
linear=nn.Linear(1,1,bias=False)
net=nn.Sequential(
    linear,
    linear)
print(net)
for name,param in net.named_parameters():
    init.constant_(param,val=3)
    print(name,param)


Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight Parameter containing:
tensor([[3.]], requires_grad=True)


In [12]:
#在内存中，两个线性层是一个对象

print(id(net[0])==id(net[1]))

print(id(net[0].weight)==id(net[1].weight))

True
True


In [13]:
#因为模型参数里包含了梯度，在反向传播计算时，共享参数的梯度是累加的

x=torch.ones(1,1)
y=net(x).sum()
print('y:',y)
y.backward()
print(net[0].weight.grad)#单次梯度是3，两次就是6
print(net[1].weight.grad)

y: tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
tensor([[6.]])
